In [1]:
import sqlite3
import numpy as np
import pandas as pd
from datetime import datetime as dt, timedelta

In [2]:
airlines_df = pd.read_excel("../data/Airlines.xlsx", index_col="AirlineId")
airports_df = pd.read_excel("../data/Airports.xlsx", index_col="AirportId")

In [3]:
airports_df

,AirportCode,FullName,OriginCountry,OriginCity
AirportId,,,,
1,PDL,Ponta Delgada Joao Paulo II,Portugal,Ponta Delgada
2,FAO,Faro Airport,Portugal,Faro
3,LIS,Portela,Portugal,Lisbon
4,OPO,Franciso De Sa Carneiro,Portugal,Porto
5,SJZ,Sao Jorge Airport,Portugal,Madeira
...,...,...,...,...
97,BOS,General Edward Lawrence Logan International Ai...,United States,Boston
98,DCA,Washington Dulles International Airport,United States,Washington
99,PHL,Philadelphia International Airport,United States,Philadelphia


In [4]:
INSERT_TEMPLATE = "INSERT INTO {table_name} {columns_tuple}\n\tVALUES {values_tuple}"

In [5]:
print(INSERT_TEMPLATE.format(table_name="Airports", columns_tuple=tuple(airports_df.columns), values_tuple=tuple(airports_df.iloc[0].tolist())).replace("\'", ""))

INSERT INTO Airports (AirportCode, FullName, OriginCountry, OriginCity)
	VALUES (PDL, Ponta Delgada Joao Paulo II, Portugal, Ponta Delgada)


In [6]:
airlines_df

,Name,AirlineCode
AirlineId,,
1,Ryanair,RYR
2,Lufthansa Group,DLH
3,International Airlines Group,AAN
4,Air France-KLM,SEU
5,Easyjet,EZY
6,Wizz Air,WZZ
7,American Airlines,AAL
8,Delta Air Lines,DAL
9,Southwest Airlines,SWA


In [7]:
with open(r"../scripts/sql/populate_db.sql", "w", encoding="utf8") as f:
    for index in airlines_df.index.tolist():
        #print(INSERT_TEMPLATE.format(table_name="Airline", columns_tuple=tuple(airlines_df.columns), values_tuple=tuple(airlines_df.iloc[index - 1].tolist())).replace("\'", ""))
        f.write(INSERT_TEMPLATE.format(table_name="Airline", columns_tuple=tuple(airlines_df.columns), values_tuple=tuple(airlines_df.iloc[index -1].tolist())).replace("\'", ""))
        f.write("\n")
        
    for index in airports_df.index.tolist():
        f.write(INSERT_TEMPLATE.format(table_name="Airport", columns_tuple=tuple(airports_df.columns), values_tuple=tuple(airports_df.iloc[index - 1].tolist())).replace("\'", ""))
        f.write("\n")

In [8]:
def get_airline_info(name):
    airline = airlines_df.loc[airlines_df["Name"] == name].to_dict(orient="index")
    index = list(airline.keys())[0]
    
    return index, airline[index]["AirlineCode"]

def get_airport_info(city):
    airport = airports_df.loc[airports_df["OriginCity"] == city].to_dict(orient="index")
    index = list(airport.keys())[0]
    
    return index, airport[index]["AirportCode"]

In [71]:
def get_flight_info(departure, arrival, start, duration, airline, route_index):
    airline_id, airline_code = get_airline_info(airline)
    departure_id, departure_code = get_airport_info(departure)
    arrival_id, arrival_code = get_airport_info(arrival)
    
    intercontinental = 0 if (arrival_id <= 68 & departure_id <= 68) or (arrival_id > 68 & departure_id > 68) else 1
    if intercontinental == 1:
        duration = dt.strptime(duration, "%H:%M:%S") + clock_to_timedetla("06:00:00")
    
    return airline_id, route_index, airline_code + "001" + departure_code + arrival_code, str(duration)[-8:], np.random.randint(50, 250), intercontinental

In [72]:
def clock_to_timedetla(time):
    return timedelta(hours=int(time[0:2]), minutes=int(time[3:5]), seconds=int(time[6:8]))

In [73]:
def get_route_info(departure, arrival, start, duration, airline):
    airline_id, airline_code = get_airline_info(airline)
    departure_id, departure_code = get_airport_info(departure)
    arrival_id, arrival_code = get_airport_info(arrival)
    
    return departure_id, arrival_id, str(start), str(dt.strptime(start, "%Y-%m-%d %H:%M:%S") + clock_to_timedetla(duration))

In [74]:
get_flight_info("Rome", "New York", "2020-12-01 10:00:00", "02:20:00", "Ryanair", 1)

(1, 1, 'RYR001CIAJFK', '08:20:00', 212, 1)

In [75]:
get_route_info("Rome", "Madrid", "2020-12-01 10:00:00", "02:20:00", "Ryanair")

(18, 6, '2020-12-01 10:00:00', '2020-12-01 12:20:00')

In [76]:
def random_date_offset():
    # Date offset in range 00:00:00 to 40:00:00
    # seconds are alwyas 00 and minutes have precision up to 10 min
    return f"0{np.random.randint(5)}:{np.random.randint(6)}0:00"

def random_flight_time():
    # Random flight time, with max value of 9H and min 1H
    return f"0{np.random.randint(1, 10)}:{np.random.randint(6)}0:00"

In [157]:
routes = []
flights = []

start_date = "2020-01-01 8:00:00"
date = dt.strptime(start_date, "%Y-%m-%d %H:%M:%S")

for index in range(1, 10_001):
    departure_airport = airports_df["OriginCity"].iloc[np.random.randint(len(airports_df["OriginCity"].index))]
    arrival_airport = airports_df["OriginCity"].iloc[np.random.randint(len(airports_df["OriginCity"].index))]
    
    if arrival_airport == departure_airport:
        arrival_airport = airports_df["OriginCity"].iloc[np.random.randint(len(airports_df["OriginCity"].index))]
    
    if arrival_airport == departure_airport:
        arrival_airport = airports_df["OriginCity"].iloc[np.random.randint(len(airports_df["OriginCity"].index))]
        
    if arrival_airport == departure_airport:
        arrival_airport = airports_df["OriginCity"].iloc[np.random.randint(len(airports_df["OriginCity"].index))]
    
    date += clock_to_timedetla(random_date_offset())
    flight_time = random_flight_time()
    airline = airlines_df["Name"].iloc[np.random.randint(airlines_df.index.size)]
    
    flights.append(get_flight_info(departure_airport, arrival_airport, str(date), flight_time, airline, index))
    routes.append(get_route_info(departure_airport, arrival_airport, str(date), flight_time, airline))

In [158]:
flights_df = pd.DataFrame.from_records(flights, columns=["AirlineId", "RouteId", "FlightCode", "Duration", "Passengers", "Intercontinental"])

In [159]:
routes_df = pd.DataFrame.from_records(routes, columns=["DepartureAirportId", "ArrivalAirportId", "DepartureDateTime", "ArrivalDateTime"])

In [160]:
(routes_df["DepartureAirportId"]  == routes_df["ArrivalAirportId"]).value_counts()

False    10000
dtype: int64

In [162]:
flights_df["FlightCode"].value_counts()

DLH001STNLED    4
AAN001STNLPL    4
AAN001DTWSTN    4
ROU001RJKSTN    4
SEU001ATLNCL    4
               ..
SEU001YWGLYS    1
EZY001LPABNA    1
WZZ001DTWSTN    1
SWA001AALZRH    1
LOT001STNPHL    1
Name: FlightCode, Length: 9358, dtype: int64

In [198]:
copy_df = flights_df.copy(deep=True)

In [199]:
copy_df["FlightCode"].value_counts()

DLH001STNLED    4
AAN001STNLPL    4
AAN001DTWSTN    4
ROU001RJKSTN    4
SEU001ATLNCL    4
               ..
SEU001YWGLYS    1
EZY001LPABNA    1
WZZ001DTWSTN    1
SWA001AALZRH    1
LOT001STNPHL    1
Name: FlightCode, Length: 9358, dtype: int64

In [202]:
for code in copy_df["FlightCode"].unique().tolist():
    if copy_df.loc[copy_df["FlightCode"] == code].index.size > 1:
        slice_df = copy_df.loc[copy_df["FlightCode"] == code]
        counter = 1
        for index in slice_df.index.tolist():
            if counter > 1:
                copy_df["FlightCode"].iloc[index] = slice_df["FlightCode"][index].replace("1", str(counter))
            counter += 1

c:\users\krzysiek\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [205]:
copy_df["FlightCode"].value_counts(dropna=False)

LOT001TRNJFK    1
SEU001STNAYT    1
DAL001MSPOPO    1
WZZ002LASCHI    1
SEU001AGPPHX    1
               ..
LOT001SEASAN    1
SEU001SKGAAL    1
SWA001ATHDUS    1
AAN001MIAYVR    1
LOT001STNPHL    1
Name: FlightCode, Length: 10000, dtype: int64

In [210]:
copy_df.to_excel(r"../data/Flights.xlsx")

In [209]:
routes_df.to_excel(r"../data/Routes.xlsx")